In [44]:
# script to calculate ratios for use in TTS method 
# match rf 11 to varying BL for ratios 
# 1) RF11 UT to RF11 BL 
# 2) RF11 UT to campaign average BL 
# 3) RF11 UT to RF11, 10, 09 BL 
# author: sofia chelpon
# date created: 3-28-2020
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns
import datetime as dt
from scipy.optimize import least_squares

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [2]:
# load in both toga and awas data pickles 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_twp.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df_twp.pkl")

In [47]:
################# -------------------- TOGA  -------------------- #################

In [3]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
toga_means = pd.DataFrame()

# rf11
toga_means['UT - RF07'] = toga_df.loc[(toga_df['GGALT'] > 12000) & (toga_df['GGALT'] < 14000)
                                                          & (toga_df['Flight'] == 'RF07')].mean()
toga_means['BL - RF07'] = toga_df.loc[(toga_df['GGALT'] < 2000) 
                                      & (toga_df['Flight'] == 'RF07')].mean()
# all rf 
toga_means['BL - All RF'] = toga_df.loc[(toga_df['GGALT'] < 2000)].mean()

toga_means = toga_means.drop('GGALT').drop('GGLAT').drop('GGLON')
#toga_means.index.name = "Trace_Gas"
toga_means = toga_means.reset_index()
toga_means = toga_means.rename(columns={'index': 'Trace_Gas'})

In [4]:
# ratios for 1) flights in total and 2) individual flights 
toga_ratios = pd.DataFrame()

toga_ratios['Trace_Gas'] = toga_means['Trace_Gas']
toga_ratios['RF07_CampAvg'] = toga_means['UT - RF07']/toga_means['BL - All RF']
toga_ratios['RF07_RF07'] = toga_means['UT - RF07']/toga_means['BL - RF07']
toga_ratios

# drop species names so doesnt duplicate column when merged 
toga_ratios = toga_ratios.drop('Trace_Gas', axis = 1)

In [5]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_12162019.xlsx'
toga_lifetimes = pd.read_excel(mypath2)

# merge lifetimes to ratios
toga_ratios_full = toga_lifetimes.merge(toga_ratios, left_index=True, right_index=True)
toga_ratios_full.insert(0, 'Instrument', 'TOGA')

In [6]:
toga_ratios_full

,Instrument,Trace_Gas,BL_tau,TROPO_tau,UT_tau,RF07_CampAvg,RF07_RF07
0,TOGA,Acetaldehyde,0.289063,0.365920,0.376967,-0.210601,-0.241412
1,TOGA,n_Butane,1.959376,3.813463,8.057685,0.184341,0.151014
2,TOGA,i_Butane,2.175570,4.031311,7.903592,0.364254,0.487147
3,TOGA,Benzene,3.755136,6.447323,11.947207,0.470292,0.563481
4,TOGA,C3H8,4.214557,9.062693,22.450000,0.643651,0.629700
5,TOGA,Methanol,5.160001,9.567463,20.040000,0.730230,0.654727
6,TOGA,Bromoform,15.069605,23.166062,33.432173,0.415254,0.350427
7,TOGA,Tetrachloroethylene,27.471720,77.860538,317.837040,0.792011,0.687468
8,TOGA,Dibromomethane,39.129546,95.533111,306.353687,0.800066,0.729214
9,TOGA,Dichloromethane,45.867749,109.800998,343.676023,1.013401,0.989712


In [52]:
################# -------------------- AWAS  -------------------- #################

In [7]:
# get mean in UT and BL for 1) flights in total and 2) individual flights 
awas_means = pd.DataFrame()

# rf08
awas_means['UT - RF07'] = awas_df.loc[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)
                                                          & (awas_df['Flight'] == 'RF07')].mean()
awas_means['BL - RF07'] = awas_df.loc[(awas_df['GGALT'] < 2000) 
                                      & (awas_df['Flight'] == 'RF07')].mean()
# all rf 
awas_means['BL - All RF'] = awas_df.loc[(awas_df['GGALT'] < 2000)].mean()

awas_means = awas_means.drop('GGALT').drop('GGLAT').drop('GGLON')
#toga_means.index.name = "Trace_Gas"
awas_means = awas_means.reset_index()
awas_means = awas_means.rename(columns={'index': 'Trace_Gas'})

In [8]:
# ratios for 1) flights in total and 2) individual flights 
awas_ratios = pd.DataFrame()

awas_ratios['Trace_Gas'] = awas_means['Trace_Gas']
awas_ratios['RF07_CampAvg'] = awas_means['UT - RF07']/awas_means['BL - All RF']
awas_ratios['RF07_RF07'] = awas_means['UT - RF07']/awas_means['BL - RF07']

# drop species names so doesnt duplicate column when merged 
awas_ratios = awas_ratios.drop('Trace_Gas', axis = 1)

In [9]:
### add attribute to each species (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_12162019.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
len(awas_lifetimes)

# merge lifetimes to ratios 
awas_ratios_full = awas_lifetimes.merge(awas_ratios, left_index=True, right_index=True)
awas_ratios_full.insert(0, 'Instrument', 'AWAS')

In [56]:
################# -------------------- MERGE THE TWO  -------------------- #################

In [10]:
master_list = awas_ratios_full
master_list = master_list.append(toga_ratios_full, ignore_index = True, sort = False)

# pickle this!! 
master_list.to_pickle("./contrast_ratios_rf07.pkl")

# look at output 
#master_list[master_list['Trace_Gas'] == 'Dichloromethane'].drop('BL_tau', axis = 1).drop('UT_tau', axis = 1)